https://stackoverflow.com/questions/61928923/dash-app-call-back-append-new-entry-in-data-frame-and-replace-if-matches-with-an

In [1]:
import dash
from dash import dcc
from dash import html
from dash import dash_table

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from dash import Dash, Input, Output, State, callback

from jupyter_dash import JupyterDash

import dash_bootstrap_components as dbc

COMPONENT_STYLE = "/assets/my_component.css"
external_stylesheets=[dbc.themes.BOOTSTRAP]

app = JupyterDash(__name__,requests_pathname_prefix="/dash1/",routes_pathname_prefix='/dash1/',
                  external_stylesheets=external_stylesheets,
                  meta_tags=[{'name': 'viewport', 'content': 'width=device-width, initial-scale=1'}],
                  prevent_initial_callbacks=True,## added
                 suppress_callback_exceptions=True)
# Create server variable with Flask server object for use with gunicorn
server = app.server

In [2]:

user_key = 'access_token'
# Setup table.
columns = ['report_name', 'ad_account', 'app_id', 'access_token']
table = dash_table.DataTable(columns=[{"name": column, "id": column} for column in columns], data=[],
                             id="data_table")
# Create app.
app.layout = html.Div([dcc.Input(id=column, value=column) for column in columns] +
                      [html.Button("Save", id="save"), dcc.Store(id="cache", data=[]), table])



In [3]:

@app.callback(Output("data_table", "data"), [Input("save", "n_clicks")], [State("data_table", "data")] +
              [State(column, "value") for column in columns])
def append(n_clicks, data, *args):
    record = {columns[i]: arg for i, arg in enumerate(list(args))}
    
    print(record)
    print('---------')
    print({columns[i]: arg for i, arg in enumerate(list(args))})
    print('==================')
    # If the record (identified by user_key) already exists, update it.
    try:
        record_index = [record[user_key] for record in data].index(record[user_key])
        data[record_index] = record
    # Otherwise, append it.
    except ValueError:
        data.append({columns[i]: arg for i, arg in enumerate(list(args))})
    # Return the updated data.
    return data




In [4]:
app.run_server(host="0.0.0.0", port=5051,debug=True)
#app.run_server(debug=True,port=5051)

Dash app running on http://0.0.0.0:5051/dash1/
{'report_name': 'report_name', 'ad_account': 'ad_account', 'app_id': 'app_id', 'access_token': 'access_token1'}
---------
{'report_name': 'report_name', 'ad_account': 'ad_account', 'app_id': 'app_id', 'access_token': 'access_token1'}
